In [1]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari
from datetime import datetime

In [2]:
# path to wherever the s2p-lbm repository is cloned on your computer
repo_path = os.path.dirname(os.path.abspath(""))
os.chdir(repo_path)

from suite3d.job import Job
from suite3d import lbmio, utils, ui
from suite3d import tiff_utils as tfu

In [3]:
results_dir = '/mnt/md0/s3d-benchmarking/results/corr_map'

In [4]:
job = Job('/mnt/md0/s3d-benchmarking','benchmark-corr_map',create=False, verbosity = 3)

   Found and loaded params from /mnt/md0/s3d-benchmarking/s3d-benchmark-corr_map/params.npy


In [5]:
# %%time
# job.register_gpu()

In [6]:
mov = job.get_registered_movie('registered_fused_data', 'fused')

In [7]:
job.params['conv_filt_type'] = 'gaussian'
job.params['conv_filt_xy'] = 1.0
job.params['conv_filt_z'] = 1.0
job.params['npil_filt_type'] = 'unif'
job.params['npil_filt_xy'] = 5.0
job.params['npil_filt_z']=  1.5

job.params['sdnorm_exp']= 0.8
job.params['intensity_thresh'] : 0.2

job.params['t_batch_size'] = 300
job.params['n_proc_corr'] = 70
job.params['mproc_batchsize'] = 5


crop = ((0,18), (100,1100), (100, 950))
mov_crop = mov[crop[0][0]:crop[0][1], :,crop[1][0]:crop[1][1], 
               crop[2][0]:crop[2][1]]

In [8]:
%%time
tic = time.time()
(vmap, mean_img, max_img), __, __ = job.calculate_corr_map(mov = mov)
toc = time.time()

   Updated main params file
      Found dir /mnt/md0/s3d-benchmarking/s3d-benchmark-corr_map/mov_sub with tag mov_sub
   Saving mov_sub to /mnt/md0/s3d-benchmarking/s3d-benchmark-corr_map/mov_sub
   Using conv_filt: gaussian, 1.00, 1.00
   Using np_filt: unif, 1.50, 5.00
   Created files and dirs for 1 batches
      Running batch 1 of 1
         Will process 300 frames (0-300, t_batch_size: 300)
      Loaded and swapped, idx 0 to 300
      Calculating corr map
         Rolling mean filter
         Stdev over time
         Sharr creation
         Sub and conv
         Vmap
   Calculated corr map in 303.57 seconds
      Saving to /mnt/md0/s3d-benchmarking/s3d-benchmark-corr_map/iters/batch0000
CPU times: user 1min 47s, sys: 3min 10s, total: 4min 58s
Wall time: 5min 40s


In [9]:
timings = {'corr_map'    : toc - tic}
outputs = {'vmap'        : vmap,
           'mean_img'    : mean_img}
repo_status = utils.get_repo_status(repo_path)

In [10]:
# this updates the baseline
# utils.save_benchmark_results(results_dir, outputs, timings, repo_status, is_baseline=True)

In [11]:
utils.benchmark(results_dir, outputs, timings, repo_status)

                                         Baseline        |      Current         | 
Branch:                             dev                  | dev                  | 
Last commit hash:                   4f756530fe7b1a8b8a8b | b5a9d9cbd25ef64fb9f9 | 
Last commit summ:                   benchmarks for init, | update demo          | 
Dirty :                             True                 | True                 | 
    
Timings (s) 
corr_map                                         395.132 |              340.661 | 

Outputs: 
vmap                                                     |                      |  mismatch: 0 / 36835470 (100.00000 % match) 
           shape:                   (30, 1239, 991)      | (30, 1239, 991)      | 
           mean:                                   2.053 |                2.053 |
           std:                                    0.842 |                0.842 | 
mean_img                                                 |                      |  mismatch: 0 / 3